提交log

| 次数 | 提交时间 | 任务一模型 | 任务二模型 | 改进 |
| :---: | :---:| :---: | :---: | :--- |
| 第一次 | 2021-09-25 02:05:01 | KNN | 简单线性回归 | |
| 第二次 | 2021-10-08 01:39:11 | KNN | 简单线性回归 | ①根据tag_list,用KMeans将app分为20类②定义pre_index衡量用户的对应用的偏好 |
| 第三次 | 2021-10-10 22:13:19 | KNN | 多元线性回归 | 将年龄预测模型改为基于梯度下降的多元线性回归 |

# 1. 预处理

## 1.1 读入文件

In [2]:
#import
import pandas as pd
import numpy as np

In [2]:
# setting
pd.set_option("display.max_rows",None)

In [3]:
root_path = 'F:/代码仓/python/讯飞_移动设备用户年龄和性别预测/移动设备用户年龄和性别预测_数据集/'
# read test files
test_file_name = "test.csv"
test_file_path = root_path + test_file_name

data_test = pd.read_csv(test_file_path)

test_events_file_name = "test_app_events.csv"
test_events_file_path = root_path + test_events_file_name

data_test_events = pd.read_csv(test_events_file_path)

In [12]:
data_test.head()

,device_id,phone_brand,device_model
0,20000,0.034091,0.063784
1,20001,0.034091,0.063784
2,20002,0.000000,0.010811
3,20003,0.420455,0.817297
4,20004,0.022727,0.003243


In [13]:
data_test_events.head()

,event_id,app_id,is_installed,is_active,device_id,tag_list,date
0,39,1,1,1,20944,"[713, 704, 548]",1
1,39,2,1,1,20944,"[549, 710, 704, 548, 172]",1
2,39,67,1,0,20944,"[549, 713, 704, 548, 730, 756, 757, 775, 779, ...",1
3,39,68,1,0,20944,"[714, 723, 548, 704, 178]",1
4,39,69,1,0,20944,"[130, 223, 130]",1


## 1.2 数据清洗

In [4]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3306 entries, 0 to 3305
Data columns (total 3 columns):
device_id       3306 non-null int64
phone_brand     3306 non-null int64
device_model    3306 non-null int64
dtypes: int64(3)
memory usage: 77.6 KB


### 1.2.1 空值

In [5]:
data_test[data_test['device_id'].isnull()==True]

,device_id,phone_brand,device_model


无空值

### 1.2.2 重复值

In [9]:
data_test['device_id'].unique().shape[0]

3290

In [100]:
data_test[data_test['device_id'].duplicated()==True]

,device_id,phone_brand,device_model
601,20600,3,48
953,20951,1,466
961,20958,1,25
963,20959,3,60
985,20980,3,298
1150,21144,7,638
1359,21352,2,46
1365,21357,1,29
1542,21533,3,363
1933,21923,10,160


疑似存在重复行。

In [4]:
data_test.drop_duplicates(inplace=True)
data_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3290 entries, 0 to 3305
Data columns (total 3 columns):
device_id       3290 non-null int64
phone_brand     3290 non-null int64
device_model    3290 non-null int64
dtypes: int64(3)
memory usage: 102.8 KB


In [5]:
data_test = data_test.reset_index(drop=True)
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3290 entries, 0 to 3289
Data columns (total 3 columns):
device_id       3290 non-null int64
phone_brand     3290 non-null int64
device_model    3290 non-null int64
dtypes: int64(3)
memory usage: 77.2 KB


### 1.2.3 归一化

In [6]:
# 归一化函数
def min_max(x,mmin,mmax):
    return (x-mmin) / (mmax-mmin)

In [7]:
mmin = data_test['phone_brand'].min()
mmax = data_test['phone_brand'].max()
data_test['phone_brand'] = data_test['phone_brand'].apply(min_max,args=(mmin,mmax))

mmin = data_test['device_model'].min()
mmax = data_test['device_model'].max()
data_test['device_model'] = data_test['device_model'].apply(min_max,args=(mmin,mmax))

### 1.2.4 其它

In [8]:
# 序列化函数
def set_pickle(path, pk):
    import pickle
    f=open(path,'wb')
    pickle.dump(pk,f)
    f.close()

# 反序列化
def get_pickle(path):
    import pickle
    f=open(path,'rb')
    pk = pickle.load(f)
    f.close()
    return pk

对tag_list字段进行预处理，去除夹杂的双引号

In [9]:
def delet_quote(row):
    left = int(0)
    right = len(row)
    if row[0]=='\"':
        left = int(1)
    if row[-1]=='\"':
        right = int(-1)
    return row[left:right]
data_test_events['tag_list'] = data_test_events['tag_list'].apply(delet_quote)

In [11]:
data_test_events.head()

,event_id,app_id,is_installed,is_active,device_id,tag_list,date
0,39,1,1,1,20944,"[713, 704, 548]",1
1,39,2,1,1,20944,"[549, 710, 704, 548, 172]",1
2,39,67,1,0,20944,"[549, 713, 704, 548, 730, 756, 757, 775, 779, ...",1
3,39,68,1,0,20944,"[714, 723, 548, 704, 178]",1
4,39,69,1,0,20944,"[130, 223, 130]",1


## 1.3 从event数据集提取特征

In [12]:
data_test_events.describe()

,event_id,app_id,is_installed,is_active,device_id,date
count,1.739053e+06,1.739053e+06,1739053.0,1.739053e+06,1.739053e+06,1.739053e+06
mean,1.628116e+06,6.414748e+02,1.0,3.798740e-01,2.161487e+04,4.021011e+00
std,9.390069e+05,1.214213e+03,0.0,4.853554e-01,9.361960e+02,1.974944e+00
min,3.900000e+01,0.000000e+00,1.0,0.000000e+00,2.000000e+04,0.000000e+00
25%,8.135440e+05,7.200000e+01,1.0,0.000000e+00,2.081200e+04,2.000000e+00
50%,1.628166e+06,2.150000e+02,1.0,0.000000e+00,2.163800e+04,4.000000e+00
75%,2.444281e+06,6.520000e+02,1.0,1.000000e+00,2.245400e+04,6.000000e+00
max,3.252926e+06,1.373600e+04,1.0,1.000000e+00,2.328900e+04,8.000000e+00


### 1.3.1 提取用户活跃系数

用户活跃系数由$c_i$和$a_i$来描述，其中连续因子$c_i$表示设备连续活跃天数（中断则重新计算），$a_i$表示设备第i天的活跃次数。则活跃系数$A$由以下公式定义：

$$A=\frac{1}{N}\sum_{i=0}^Na_ic_i$$

活跃系数将作为特征字段active_index存入训练数据表。由于尺度可能会偏大，所以考虑做归一化处理。

In [14]:
data_test_events_gb = data_test_events.groupby(['device_id','date'])['event_id'].count()
# data_test_events_gb

下面提取每台设备的活跃系数，在训练集中插入为active_index：

In [15]:
c = [0 for i in range(0,9)] #连续系数
a = [0 for i in range(0,9)] #活跃次数
A = np.array([0 for i in range(0,data_test['device_id'].unique().shape[0])])
for i,di in enumerate(data_test['device_id'].unique()):
# for i in range(0,data_test['device_id'].unique().shape[0]):
    active_date_ls = []
    if isinstance(data_test_events_gb[di],int):
        active_date_ls = [data_test_events_gb[di]]
    elif isinstance(data_test_events_gb[di],np.int64):
        active_date_ls = [int(data_test_events_gb[di])]
    else:
        active_date_ls = data_test_events_gb[di].index.tolist()
    ac_date = [1 if d in active_date_ls else 0 for d in range(0,8)]
    
    # 连续系数
    ct = 0 #连续次数
    for j in range(1,8):
        if ac_date[j]==1 and ac_date[j-1]==1:
            ct+=1.0
        elif ac_date[j]==1:
            ct = 1.0
        else:
            ct = 0
        c[j] = 1.0+0.1*ct
    
#   每日事件数  ai = data_train_events_gb[device_id,day]
    a_d = [data_test_events_gb[di,d] for d in active_date_ls]  
    for d in range(0,len(active_date_ls)):
        a[active_date_ls[d]] = a_d[d]
    
    A[i] = np.sum(np.array(a)*np.array(c))
    
# active_index = ai*ci
data_test['active_index'] = pd.Series(A)
data_test.head()

,device_id,phone_brand,device_model,active_index
0,20000,0.034091,0.063784,471
1,20001,0.034091,0.063784,354
2,20002,0.000000,0.010811,1607
3,20003,0.420455,0.817297,1005
4,20004,0.022727,0.003243,2495


对active_index做一个归一化

In [16]:
mmin = data_test['active_index'].min()
mmax = data_test['active_index'].max()
data_test['active_index'] = data_test['active_index'].apply(min_max,args=(mmin,mmax))

In [17]:
print(mmin,mmax)

59 59293


### 1.2.2 提取用户偏好

#### 1.2.2.1 偏好系数

定义偏好系数pre_index如下：

$$pre\_index=\sum (is\_installed+1.5is\_active)$$

In [18]:
def generate_preIndex(row):
    is_installed = data_test_events[data_test_events['device_id']==row]['is_installed']
    is_active = data_test_events[data_test_events['device_id']==row]['is_active']
#     print((is_installed + 1.5 * is_active).sum())
    return (is_installed + 1.5 * is_active).sum()

In [19]:
data_test['pre_index'] = data_test['device_id'].apply(generate_preIndex)
data_test['pre_index'].describe()

count     3290.000000
mean       829.782523
std       2082.793402
min          3.500000
25%        111.000000
50%        312.000000
75%        852.250000
max      59742.500000
Name: pre_index, dtype: float64

从标准差可以看出，不同设备之间的偏好系数差别很大，数值越大越离散。

对pre_index进行标准化：

In [20]:
mmin = data_test['pre_index'].min()
mmax = data_test['pre_index'].max()
data_test['pre_index'] = data_test['pre_index'].apply(min_max,args=(mmin,mmax))
data_test.head()

,device_id,phone_brand,device_model,active_index,pre_index
0,20000,0.034091,0.063784,0.006955,0.010178
1,20001,0.034091,0.063784,0.004980,0.005842
2,20002,0.000000,0.010811,0.026134,0.020322
3,20003,0.420455,0.817297,0.015971,0.001984
4,20004,0.022727,0.003243,0.041125,0.074750


#### 1.2.2.2 各类型app的偏好情况

使用预训练的KMeans模型，根据tag_list将app分为20类。通过偏好系数来衡量用户对某类app的偏好程度。

首先载入KMeans模型：

In [21]:
mbk = get_pickle('KNN_appType_completed.pkl')

In [22]:
'''
功能：1.发现app_type 2.在X的基础上添加每类app的偏好系数
X：数据集，array
return：X_1，array
'''
def pre_treatment(X, quick_pattern = True):    
    # 将tag_list转换为特征矩阵
#     array_tag = get_arr_tag(X)
    # 创建array
    device_id_list = list(tuple(int(i)for i in X[:,0]))

    # 取出训练集范围的标签列表
    af = data_test_events[data_test_events['device_id'].isin(device_id_list)].reset_index(drop=True)

    w = 26
    h = af.shape[0]
    array_tag = np.zeros([h,w])

    # tag_list标签最值
    mmin = 1
    mmax = 1021
    
    # 获取app特征矩阵
    for i in range(0,af.shape[0]):
        tag_list = af.loc[i,'tag_list']
        r = tag_list[1:-1].split(',')
        li = [min_max(int(s),mmin,mmax) for s in r] #归一化
        for j in range(0,len(li)):
            array_tag[i][j] = li[j]
        
    # 生成app_type
    n_clusters = 20
    random_state = 17
    batch_size = 100

    mbm_pred = mbk.predict(array_tag)
    af['app_type'] = pd.Series(mbm_pred)
    
    
    # 生成每种app的偏好系数
    af['pre_index'] = af['is_installed'] + 1.5 * af['is_active']
    pre_index_sum = af.groupby(['device_id','app_type'])['pre_index'].agg(np.sum)

    t_arr = np.zeros([X.shape[0],20])
    for i in range(0,X.shape[0]):
        did = int(X[i][0])
        for j_i,j in enumerate(list(pre_index_sum[did].index)):
            t_arr[i][j] = pre_index_sum[did].values[j_i]

    # 归一化
    for i in range(0,t_arr.shape[0]):
        t_arr[i,:] = (t_arr[i,:] - np.min(t_arr[i,:])) / (np.max(t_arr[i,:]) - np.min(t_arr[i,:]))

    # 合并
    X_1 = np.c_[X,t_arr]

    # 删除第一列
    X_1 = np.delete(X_1, 0, axis=1)
    X_1[0,:]
    
    return X_1

In [23]:
X_1 = pre_treatment(data_test.values)

# 2 任务一：性别预测

性别是标量，做个二分类就行。

训练时发现在当前任务中，KNN的表现比决策树较好，因此使用KNN模型对性别进行预测。

导入模型：

In [30]:
from sklearn.neighbors import KNeighborsClassifier

In [25]:
# 读取
knn = get_pickle('KNeighborsClassifier.model')

预测：

In [26]:
# knn_pred = knn.predict(X_holdout)
knn_pred = knn.predict(X_1)

## 2.2 年龄预测模型

### 2.2.1 简单线性回归

导入模型：

In [27]:
from sklearn.linear_model import LinearRegression

In [38]:
lin_reg = get_pickle('LinearRegression.model')

预测：

In [49]:
from sklearn.metrics import accuracy_score,mean_absolute_error

In [31]:
# linreg_pred = lin_reg.predict(X_holdout)
linreg_pred = lin_reg.predict(X_1)
linreg_pred

array([30.10944992, 32.43495907, 29.00769376, ..., 30.11350357,
       33.83719556, 32.07568993])

### 2.2.2 多元线性回归

In [27]:
# import
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

导入模型：

In [39]:
regressor = get_pickle('SGDRegressor.model')

对测试集进行Z-score标准化：

In [40]:
X_scaler = StandardScaler()
X_scaler.fit(X_1)
X_test_scaled = X_scaler.transform(X_1)

预测：

In [30]:
reg_pred_scaled = regressor.predict(X_test_scaled)
reg_pred_scaled

array([-0.1271872 , -0.09206262, -0.34765111, ..., -0.21018585,
        0.07282458, -0.01031541])

将预测结果映射回原先的取值域：

In [34]:
y_scaler = get_pickle('y_scaler.model')

In [35]:
np.round(y_scaler.inverse_transform(reg_pred_scaled))

array([31., 32., 29., ..., 30., 33., 32.])

刚开始使用的是标签标准化后训练得到的模型，但测试集和训练集的值域毕竟不同，使用训练集的ScaleStandard去还原预测数据可能会有问题，所以又试了未标准化标签的模型（如下），但是最终得分居然差不多。

In [ ]:
regressor = get_pickle('SGDRegressor.model')

X_scaler = StandardScaler()
X_scaler.fit(X_1)
X_test_scaled = X_scaler.transform(X_1)

reg_pred = regressor.predict(X_test_scaled)
reg_pred

## 3 导出结果

In [36]:
def generate_submit_file(gender_pred, age_pred, path):
    c = np.array([np.array(data_test['device_id'].tolist()),gender_pred,np.round(age_pred)]).T
    out = pd.DataFrame(c,columns=['device_id','gender','age'])
    
    out['device_id'] = out['device_id'].astype('int64')
    out['gender'] = out['gender'].astype('int64')
    out['age'] = out['age'].astype('int64')
    
    out.to_csv(path,index=False)

In [42]:
out_fn = 'submit.csv'
# generate_submit_file(knn_pred,np.round(y_scaler.inverse_transform(reg_pred_scaled)),root_path + out_fn)
generate_submit_file(knn_pred,np.round(reg_pred),root_path + out_fn)